# mssql_to_raw

Moves data from SQL Server into Databricks Delta tables. Data will firstly be stored in the raw zone, where it is kept "as-it-is". You can change the raw zone catalog in Unity Catalog by setting the **raw_catalog_name** job parameter.

In [ ]:
import sys, os
from datetime import datetime, timedelta

sys.path.append(os.path.abspath("../"))

run_date = datetime.strptime(dbutils.widgets.get("run_date"), '%Y-%m-%d')
start_date = run_date - timedelta(days=1)
secret_scope = dbutils.widgets.get("secret_scope")
ctr_catalog_name = dbutils.widgets.get("ctr_catalog_name")
raw_catalog_name = dbutils.widgets.get("raw_catalog_name")

In [ ]:

from lib.el import MSSqlDataLoader, SqlConnectionData
from lib.logging import Log, Logger

#Sets data necessary to connect to SQL Server
connection_data = SqlConnectionData(
    host = dbutils.widgets.get("mssql_host"),
    port = dbutils.widgets.get("mssql_port"),
    database = dbutils.widgets.get("mssql_database"),
    username = dbutils.secrets.get(secret_scope, "mssql_username"),
    password = dbutils.secrets.get(secret_scope, "mssql_password")
)

#Selects the tables that will be ingested
df_data_ingestion = (spark.read
    .table(f"{ctr_catalog_name}.loading.data_ingestion")
    .select(
        "schema_name", 
        "table_name", 
        "primary_key", 
        "filter",
        "selected",
        "partition_column",
        "num_partitions")
    .where("active = true")
)

logs = []
processing_has_failed = False
#Sets the filter to be used with incremental loads
start_date_filter = f"CONVERT(DATETIME, '{start_date.strftime('%Y-%m-%d')}', 120)"

for row in df_data_ingestion.collect():
    schema_name = row["schema_name"]
    table_name = row["table_name"]
    partition_column = row["partition_column"]
    num_partitions = row["num_partitions"]
    primary_key = row["primary_key"].replace(" ", "").split(",")
    selected = None if row["selected"] is None else row["selected"].replace(" ", "").split(",")

    data_loader = MSSqlDataLoader(
        schema_name = schema_name, 
        table_name = table_name, 
        primary_key = primary_key,
        connection_data = connection_data,
        selected = selected,
        num_partitions = num_partitions,
        partition_column = partition_column
    )

    table_full_name = f"{raw_catalog_name}.{schema_name}.{table_name}"
    
    log = Log(
        target_catalog_name = raw_catalog_name, 
        target_schema_name = schema_name,
        target_table_name = table_name, 
        source_catalog_name = None, 
        source_schema_name = schema_name, 
        source_table_name = table_name
    )

    try:
        #If the table exists, loads it incrementally, else loads the full table
        if spark.catalog.tableExists(table_full_name):
            filter = row["filter"].replace(":start_date", start_date_filter)
        else:
            filter = None

        #Extracts data from SQL Server and loads it into the raw zone
        data_loader.extract(filter).load_into(table_full_name)
        log.movements = data_loader.df.count()
    except Exception as e:
        log.error = repr(e)
        processing_has_failed = True
        
    logs.append(log)

logger = Logger(f"{ctr_catalog_name}.loading.data_ingestion_log")

logger.log(logs)

if processing_has_failed:
    raise RuntimeError("Processing of one or more tables has failed. Check the data ingestion log for further info.")